In [1]:
import polars as pl
# 讀取 CSV 檔案
df = pl.read_csv("amazon_2023.csv")

### 整理評論相關欄位:
- title_length: 標題長度
- text_length: 文本長度
- user_avg_text_length: 該用戶所有評論的平均文本長度
- user_review_count: 該用戶的評論總數
- days_since_review_ln : 評論發布距今天數

In [14]:
# 處理評論數據並計算基本屬性
processed_df = df.with_columns([
    # 1. 計算 title 長度
    pl.col("title").str.len_chars().alias("title_length"),
    
    # 2. 計算 text 長度
    pl.col("text").str.len_chars().alias("text_length"),
    
    # 5. 計算評論發布距今天數（自然對數）
    # unix timestamp 轉換為天數差異再取對數
    (pl.lit(1750089600000) - (pl.col("timestamp")) / 86400000).log().alias("days_since_review_ln")
]).with_columns([
    # 3. 計算每個 user_id 的平均 text 長度
    pl.col("text_length").mean().over("user_id").alias("user_avg_text_length"),
    
    # 4. 計算每個 user_id 的評論總數
    pl.col("user_id").count().over("user_id").alias("user_review_count")
]).with_columns([
    # 取自然對數處理
    pl.col("title_length").log().alias("title_length_ln"),
    pl.col("text_length").log().alias("text_length_ln"),
    pl.col("user_avg_text_length").log().alias("user_avg_text_length_ln"),
    pl.col("user_review_count").log().alias("user_review_count_ln")
])


In [15]:
processed_df.head(1)

rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,main_category,average_rating,rating_number,features,description,price,videos,store,categories,details,bought_together,subtitle,author,title_length,text_length,days_since_review_ln,user_avg_text_length,user_review_count,title_length_ln,text_length_ln,user_avg_text_length_ln,user_review_count_ln
f64,str,str,str,str,str,str,i64,i64,bool,str,f64,f64,str,str,str,str,str,str,str,str,str,str,u32,u32,f64,f64,u32,f64,f64,f64,f64
1.0,"""Watercolor with Me in the Ocea…","""It is definitely not a waterco…","""{'hi_res': [None], 'large': ['…","""B09BGPFTDB""","""B09BGPFTDB""","""AFKZENTNBQ7A7V7UXW5JJI6UGRYQ""",1642399598485,0,true,"""Books""",1.0,1.0,"""['This incredible adult colori…","""[]""","""5.99""","""{'title': [], 'url': [], 'user…","""LEONARD DRAVEN (Author)""","""['Books', 'Arts & Photography'…","""{""Publisher"": ""Independently p…",null,"""Paperback – July 29, 2021""",null,56,1427,28.190688,731.173913,23,4.025352,7.26333,6.594651,3.135494


### 標準化並淨化文本數據
1. 將所有單詞轉換為小寫 (transforming all words into lowercase)
2. 刪除所有停用詞、標點符號、數位和空格 (removing all stop words, punctuation marks, numbers, and spaces)
3. 對單詞進行詞幹提取（例如，values、valued 和 valuing 都替換為 value）(stemming words)

In [17]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import nltk

# 下載必要的NLTK資源
nltk.download('stopwords', quiet=True)

def preprocess_text(text):
    """
    文本預處理函數
    1. 轉換為小寫
    2. 刪除停用詞、標點符號、數字和多餘空格
    3. 詞幹提取
    """
    if pd.isna(text):
        return ""
    
    # 步驟1: 轉換為小寫
    text = text.lower()
    
    # 步驟2: 刪除標點符號和數字，只保留字母和空格
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # 分割單詞並刪除停用詞
    stop_words = set(stopwords.words('english'))
    words = text.split()
    words = [word for word in words if word and word not in stop_words]
    
    # 步驟3: 詞幹提取
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]
    
    # 移除空字符串並重新組合
    return ' '.join([word for word in words if word])

# 讀取CSV文件
df = pd.read_csv("amazon_2023.csv")

# 對title和text欄位進行預處理
df['title_processed'] = df['title'].apply(preprocess_text)
df['text_processed'] = df['text'].apply(preprocess_text)

# 保存處理後的結果
df.to_csv("amazon_2023_processed.csv", index=False)

print("文本預處理完成！")
print(f"處理了 {len(df)} 筆資料")
print("\n預處理前後對比範例:")
print("=" * 50)
if len(df) > 0:
    print("原始title:", df['title'].iloc[0][:100] + "..." if len(str(df['title'].iloc[0])) > 100 else df['title'].iloc[0])
    print("處理後title:", df['title_processed'].iloc[0][:100] + "..." if len(str(df['title_processed'].iloc[0])) > 100 else df['title_processed'].iloc[0])
    print("\n原始text:", df['text'].iloc[0][:100] + "..." if len(str(df['text'].iloc[0])) > 100 else df['text'].iloc[0])
    print("處理後text:", df['text_processed'].iloc[0][:100] + "..." if len(str(df['text_processed'].iloc[0])) > 100 else df['text_processed'].iloc[0])

c:\Users\ChenChun\.conda\envs\myenv\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
C:\Users\ChenChun\AppData\Local\Temp\ipykernel_26844\3043388951.py:39: DtypeWarning: Columns (15,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("amazon_2023.csv")


文本預處理完成！
處理了 409482 筆資料

預處理前後對比範例:
原始title: Watercolor with Me in the Ocean: Coloring book for Adult
處理後title: watercolor ocean color book adult

原始text: It is definitely not a watercolor book.  The paper bucked completely.  The pages honestly appear to ...
處理後text: definit watercolor book paper buck complet page honestli appear photo copi pictur say bc look seal p...


### Emotional Feature Extraction
在文本預處理之後，研究使用特定的字典來量化每條評論中的情感效價 (valence) 和喚醒度 (arousal)。<br>
使用預先量化的情感字典：該研究使用了 Warriner 等人 開發的字典，該字典量化了近 14,000 個英文單詞的情感效價和喚醒度分數。這種方法超越了僅僅計算正面或負面詞彙的數量，能夠更好地捕捉情感的強度。

1. 建立 Valence 欄位
A review's overall valence measured by the difference between scores of positive words and negative words
<br>評價的總體效價由正面詞和負面詞的分數之間的差異來衡量

2. 建立 Arousal 欄位
The level of emotion activation measured by scores of arousal words in a review.
<br>通過評論中喚醒詞的分數來衡量的情緒激活水準。

In [ ]:
# 載入數據
df = pd.read_csv("amazon_2023_processed.csv")
emotion_dict = pd.read_csv("springer_data/BRM-emot-submit.csv")

# 建立詞彙-情感分數映射
word_scores = {}
for _, row in emotion_dict.iterrows():
    word_scores[str(row['Word']).lower()] = {
        'valence': row['V.Mean.Sum'],
        'arousal': row['A.Mean.Sum']
    }

# 計算情感分數函數
def calculate_emotion_scores(text):
    if pd.isna(text):
        return 0, 0
    
    words = str(text).lower().split()
    
    # 計算效價和喚醒度
    positive_scores = []
    negative_scores = []
    arousal_scores = []
    
    for word in words:
        word = word.strip('.,!?";:()[]{}')
        if word in word_scores:
            valence = word_scores[word]['valence']
            arousal = word_scores[word]['arousal']
            
            if valence > 5:
                positive_scores.append(valence)
            elif valence < 5:
                negative_scores.append(valence)
            
            arousal_scores.append(arousal)
    
    # 效價 = 正面分數總和 - 負面分數總和
    valence_score = sum(positive_scores) - sum(negative_scores)
    # 喚醒度 = 所有喚醒度分數總和
    arousal_score = sum(arousal_scores)
    
    return valence_score, arousal_score

# 計算每條評論的情感分數
valence_scores = []
arousal_scores = []

for _, row in df.iterrows():
    # 合併標題和文本
    combined_text = str(row['title_processed']) + " " + str(row['text_processed'])
    valence, arousal = calculate_emotion_scores(combined_text)
    valence_scores.append(valence)
    arousal_scores.append(arousal)

# 添加新欄位
df['Valence'] = valence_scores
df['Arousal'] = arousal_scores

# 顯示統計結果
print(f"效價分數 - 平均值: {df['Valence'].mean():.3f}, 標準差: {df['Valence'].std():.3f}")
print(f"喚醒度分數 - 平均值: {df['Arousal'].mean():.3f}, 標準差: {df['Arousal'].std():.3f}")

# 保存結果
df.to_csv("amazon_2023_with_emotions.csv", index=False)
print("結果已保存到 amazon_2023_with_emotions.csv")

### 建立 廣度(Breadth)欄位
The number of topics of a review obtained from NMF topic modeling.
<br>從 NMF 主題建模獲得的評價的主題數。

#### 計算「廣度」（Breadth）的步驟

1.  **主題建模（Topic Modeling）**
    * 研究利用**非負矩陣分解（Non-Negative Matrix Factorization, NMF）**技術，自動從所有評論中發現共同主題。

2.  **生成權重矩陣 W**
    * NMF 會將評論的文本資料分解為兩個非負矩陣 $K$ 和 $W$ 的乘積。
    * $W$ 矩陣是一個 $k$（主題數量）乘以 $n$（評論數量）的**權重矩陣**。
    * $W$ 矩陣中的每個條目 $w_{ij}$ 代表**主題 $i$ 對於評論 $j$ 的權重**。這些 $w_{ij}$ 可被視為評論在特定主題上的「廣度分數」或「權重」。

3.  **判斷主題是否被涵蓋**
    * 為了確定每則評論的「廣度」，研究會評估每篇評論所涵蓋的總主題數量。
    * 判斷一個主題 $i$ 是否被評論 $j$ 涵蓋的標準是：如果該評論 $j$ 在主題 $i$ 上的權重 ($w_{ij}$) **大於**該主題 $i$ 在所有評論中權重的中位數 ($median(w_{i1},...,w_{in})$)。

4.  **計算評論的「廣度」**
    * 一篇評論的「廣度」（Breadth）被定義為所有主題中，**符合上述條件（即 $w_{ij}$ 超過中位數）的主題數量之和**。
    * 這透過一個**二元指示函數（Ind）**來計算，其公式為：
        $$Breadth_j = \sum_{i=1}^{k} Ind(w_{ij} > median(w_{i1},...,w_{in}))$$
    * 這裡的 $Ind$ 函數，如果條件為真（即 $w_{ij}$ 大於中位數），則返回 1，否則返回 0。



In [33]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

def nmf_breadth_analysis(df, n_topics=10):
    """
    執行NMF廣度分析
    
    參數:
    df: DataFrame，包含'title_processed'和'text_processed'欄位
    n_topics: 主題數量，預設為10
    
    返回:
    df: 新增'breadth'欄位的DataFrame
    """
    
    # 合併標題和文本（已經過預處理）
    df['combined_text'] = df['title_processed'].fillna('') + ' ' + df['text_processed'].fillna('')
    
    # 移除空文本，但保留原始索引
    mask = df['combined_text'].str.len() > 0
    df_clean = df[mask].copy()
    
    if len(df_clean) == 0:
        print("警告：沒有有效的文本資料進行分析")
        df['breadth'] = 0
        return df
    
    # 建立文件-詞項矩陣 (TF-IDF)
    vectorizer = TfidfVectorizer(max_features=1000, min_df=2, max_df=0.95)
    D = vectorizer.fit_transform(df_clean['combined_text'])
    
    # 執行NMF分解
    # D矩陣: m(詞彙) × n(評論)
    # K矩陣: m(詞彙) × k(主題) - 詞彙-主題矩陣
    # W矩陣: k(主題) × n(評論) - 主題-評論權重矩陣
    nmf = NMF(n_components=n_topics, random_state=42, max_iter=200)
    W = nmf.fit_transform(D)  # W矩陣：k(主題) × n(評論)
    K = nmf.components_      # K矩陣：k(主題) × m(詞彙)
    
    # 計算廣度 - 按照公式: Breadth_j = ∑k i=1 Ind(wij > median(wi1,…,win))
    breadth_scores = []
    
    # 預先計算每個主題的中位數
    topic_medians = []
    for i in range(W.shape[0]):  # 對每個主題i
        topic_weights = W[i, :]  # 主題i在所有評論中的權重
        median_weight = np.median(topic_weights)
        topic_medians.append(median_weight)
    
    # 計算每篇評論的廣度
    for j in range(W.shape[1]):  # 對每篇評論j
        breadth = 0
        for i in range(W.shape[0]):  # 對每個主題i
            # 二元指示函數 Ind(w_ij > median(w_i1,...,w_in))
            if W[i, j] > topic_medians[i]:
                breadth += 1  # 符合條件的主題計數+1
        
        breadth_scores.append(breadth)
    
    # 初始化廣度欄位
    df['breadth'] = 0
    
    # 逐一對應廣度分數
    for i, idx in enumerate(df_clean.index):
        df.at[idx, 'breadth'] = breadth_scores[i]
    
    return df

# 載入已預處理的資料
df = pd.read_csv("amazon_2023_with_emotions.csv")

# 執行NMF廣度分析
df_result = nmf_breadth_analysis(df, n_topics=10)

# 存檔
df_result.to_csv("amazon_2023_with_breadth.csv", index=False)

print(f"分析完成！資料已存為 amazon_2023_with_breadth.csv")
print(f"總共 {len(df_result)} 筆資料")
print(f"廣度分數統計：")
print(df_result['breadth'].describe())

C:\Users\ChenChun\AppData\Local\Temp\ipykernel_26844\4060925740.py:72: DtypeWarning: Columns (15,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("amazon_2023_with_emotions.csv")


IndexError: list index out of range

In [34]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

def nmf_breadth_analysis(df, n_topics=10):
    """
    執行NMF廣度分析
    
    參數:
    df: DataFrame，包含'title_processed'和'text_processed'欄位
    n_topics: 主題數量，預設為10
    
    返回:
    df: 新增'breadth'欄位的DataFrame
    """
    
    # 合併標題和文本（已經過預處理）
    df['combined_text'] = df['title_processed'].fillna('') + ' ' + df['text_processed'].fillna('')
    
    # 移除空文本，但保留原始索引
    mask = df['combined_text'].str.len() > 0
    df_clean = df[mask].copy()
    
    if len(df_clean) == 0:
        print("警告：沒有有效的文本資料進行分析")
        df['breadth'] = 0
        return df
    
    # 建立文件-詞項矩陣 (TF-IDF)
    vectorizer = TfidfVectorizer(max_features=1000, min_df=2, max_df=0.95)
    D = vectorizer.fit_transform(df_clean['combined_text'])
    
    print(f"有效資料數量: {len(df_clean)}")
    print(f"文件-詞項矩陣形狀: {D.shape}")
    
    if D.shape[0] == 0:
        print("錯誤：沒有有效的詞彙進行分析")
        df['breadth'] = 0
        return df
    
    # 執行NMF分解
    # D矩陣: n(評論) × m(詞彙)
    # W矩陣: n(評論) × k(主題) - fit_transform的結果
    # K矩陣: k(主題) × m(詞彙) - components_
    nmf = NMF(n_components=n_topics, random_state=42, max_iter=200)
    W = nmf.fit_transform(D)  # W矩陣：n(評論) × k(主題)
    K = nmf.components_      # K矩陣：k(主題) × m(詞彙)
    
    print(f"W矩陣形狀: {W.shape}")
    print(f"K矩陣形狀: {K.shape}")
    
    # 轉置W矩陣以符合定義：k(主題) × n(評論)
    W = W.T  # 現在W[i,j]表示主題i對評論j的權重
    print(f"轉置後W矩陣形狀: {W.shape}")
    
    # 計算廣度 - 按照公式: Breadth_j = ∑k i=1 Ind(wij > median(wi1,…,win))
    breadth_scores = []
    
    # 預先計算每個主題的中位數
    topic_medians = []
    for i in range(W.shape[0]):  # 對每個主題i
        topic_weights = W[i, :]  # 主題i在所有評論中的權重
        median_weight = np.median(topic_weights)
        topic_medians.append(median_weight)
    
    # 計算每篇評論的廣度
    for j in range(W.shape[1]):  # 對每篇評論j
        breadth = 0
        for i in range(W.shape[0]):  # 對每個主題i
            # 二元指示函數 Ind(w_ij > median(w_i1,...,w_in))
            if W[i, j] > topic_medians[i]:
                breadth += 1  # 符合條件的主題計數+1
        
        breadth_scores.append(breadth)
    
    # 初始化廣度欄位
    df['breadth'] = 0
    
    # 檢查長度是否匹配
    if len(breadth_scores) != len(df_clean):
        print(f"警告：廣度分數數量 ({len(breadth_scores)}) 與有效資料數量 ({len(df_clean)}) 不匹配")
        return df
    
    # 逐一對應廣度分數
    for i, idx in enumerate(df_clean.index):
        if i < len(breadth_scores):  # 額外安全檢查
            df.at[idx, 'breadth'] = breadth_scores[i]
    
    return df

# # 載入已預處理的資料
# df = pd.read_csv("amazon_2023_with_emotions.csv")

# 執行NMF廣度分析
df_result = nmf_breadth_analysis(df, n_topics=10)

# 存檔
df_result.to_csv("amazon_2023_with_breadth.csv", index=False)

print(f"分析完成！資料已存為 amazon_2023_with_breadth.csv")
print(f"總共 {len(df_result)} 筆資料")
print(f"廣度分數統計：")
print(df_result['breadth'].describe())

有效資料數量: 409482
文件-詞項矩陣形狀: (409482, 1000)
W矩陣形狀: (409482, 10)
K矩陣形狀: (10, 1000)
轉置後W矩陣形狀: (10, 409482)
分析完成！資料已存為 amazon_2023_with_breadth.csv
總共 409482 筆資料
廣度分數統計：
count    409482.000000
mean          4.890784
std           1.762957
min           0.000000
25%           4.000000
50%           5.000000
75%           6.000000
max          10.000000
Name: breadth, dtype: float64


### 可讀性分析 (Readability Analysis)
這是透過計算評論的Flesch閱讀易性指數 (Flesch Reading Ease Index) 來衡量其可讀性。這是一個客觀指標，與評論的情緒或主題內容無直接關係，而是關注文本的語言結構。

可讀性分析是透過計算評論的 Flesch 閱讀易性指數 (Flesch Reading Ease Index) 來衡量的<br>
其計算公式如下： 206.835 − 1.015 ( 總字數 / 總句數 ) − 84.6 ( 總音節數 / 總字數 )<br>
在這個公式中：
- 總字數 (total words)：指評論中的詞彙總數。
- 總句數 (total sentences)：指評論中的句子總數。
- 總音節數 (total syllables)：指評論中所有詞彙的音節總數。
研究指出，Flesch 閱讀易性指數的數值越高，代表該評論的可讀性越好

In [36]:
import pandas as pd
import re
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

# 更完整的NLTK資源下載
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    print("Downloading NLTK punkt tokenizer...")
    nltk.download('punkt')

try:
    nltk.data.find('tokenizers/punkt_tab')
except LookupError:
    print("Downloading NLTK punkt_tab tokenizer...")
    nltk.download('punkt_tab')

def count_syllables(word):
    """計算單詞音節數"""
    word = word.lower()
    if word == '':
        return 0
    
    # 移除標點符號
    word = re.sub(r'[^a-z]', '', word)
    if not word:
        return 0
    
    # 計算母音數量作為音節估算
    vowels = 'aeiouy'
    syllables = 0
    prev_was_vowel = False
    
    for char in word:
        is_vowel = char in vowels
        if is_vowel and not prev_was_vowel:
            syllables += 1
        prev_was_vowel = is_vowel
    
    # 特殊規則：以e結尾的單詞通常不發音
    if word.endswith('e') and syllables > 1:
        syllables -= 1
    
    # 每個單詞至少有1個音節
    return max(1, syllables)

def simple_sentence_tokenize(text):
    """簡單的句子分割（備用方案）"""
    # 使用正則表達式分割句子
    sentences = re.split(r'[.!?]+', text)
    # 移除空白句子
    sentences = [s.strip() for s in sentences if s.strip()]
    return sentences

def simple_word_tokenize(text):
    """簡單的單詞分割（備用方案）"""
    # 使用正則表達式提取單詞
    words = re.findall(r'\b[a-zA-Z]+\b', text)
    return words

def calculate_flesch_score(text):
    """
    計算Flesch閱讀易性指數
    公式: 206.835 - 1.015(總字數/總句數) - 84.6(總音節數/總字數)
    """
    if pd.isna(text) or text.strip() == '':
        return 0
    
    try:
        # 嘗試使用NLTK進行句子分割
        sentences = sent_tokenize(text)
        words = word_tokenize(text)
        # 只保留字母組成的詞
        words = [word for word in words if re.match(r'^[a-zA-Z]+$', word)]
    except:
        # 如果NLTK失敗，使用備用方案
        print("使用備用分詞方案...")
        sentences = simple_sentence_tokenize(text)
        words = simple_word_tokenize(text)
    
    total_sentences = len(sentences)
    total_words = len(words)
    
    if total_sentences == 0 or total_words == 0:
        return 0
    
    # 計算總音節數
    total_syllables = sum(count_syllables(word) for word in words)
    
    # 計算Flesch閱讀易性指數
    flesch_score = 206.835 - 1.015 * (total_words / total_sentences) - 84.6 * (total_syllables / total_words)
    
    return round(flesch_score, 2)

def add_readability_analysis(csv_file):
    """
    為CSV文件添加可讀性分析
    """
    try:
        # 載入資料
        df = pd.read_csv(csv_file)
        print(f"載入資料：{len(df)} 筆")
        
        # 合併title_processed和text_processed進行分析
        df['combined_text'] = df['title_processed'].fillna('') + ' ' + df['text_processed'].fillna('')
        
        # 計算Flesch可讀性指數
        print("正在計算Flesch可讀性指數...")
        
        # 添加進度顯示
        total_rows = len(df)
        readability_scores = []
        
        for i, text in enumerate(df['combined_text']):
            if i % 1000 == 0:  # 每1000筆顯示進度
                print(f"進度: {i}/{total_rows} ({i/total_rows*100:.1f}%)")
            
            score = calculate_flesch_score(text)
            readability_scores.append(score)
        
        df['readability'] = readability_scores
        
        # 移除臨時欄位
        df = df.drop('combined_text', axis=1)
        
        # 存檔
        output_file = "amazon_2023_with_breadth_readability.csv"
        df.to_csv(output_file, index=False)
        
        print(f"分析完成！最新資料已存為 {output_file}")
        print(f"可讀性分數統計：")
        print(df['readability'].describe())
        
        return df
        
    except FileNotFoundError:
        print(f"錯誤：找不到檔案 {csv_file}")
        return None
    except Exception as e:
        print(f"處理過程中發生錯誤：{str(e)}")
        return None

# 執行分析
if __name__ == "__main__":
    df_result = add_readability_analysis("amazon_2023_with_breadth.csv")
    
    if df_result is not None:
        # 顯示可讀性分數分布
        print("\n可讀性分數分布：")
        print("0-30: 非常困難")
        print("30-50: 困難") 
        print("50-60: 相當困難")
        print("60-70: 標準")
        print("70-80: 相當容易")
        print("80-90: 容易")
        print("90-100: 非常容易")
        
        # 統計各區間的數量
        bins = [0, 30, 50, 60, 70, 80, 90, 100]
        labels = ['非常困難', '困難', '相當困難', '標準', '相當容易', '容易', '非常容易']
        df_result['readability_level'] = pd.cut(df_result['readability'], bins=bins, labels=labels, include_lowest=True)
        
        print("\n各難度等級分布：")
        print(df_result['readability_level'].value_counts())
        
        # 顯示一些統計資訊
        print(f"\n總共處理 {len(df_result)} 筆資料")
        print(f"平均可讀性分數: {df_result['readability'].mean():.2f}")
        print(f"可讀性分數中位數: {df_result['readability'].median():.2f}")
        print(f"可讀性分數標準差: {df_result['readability'].std():.2f}")

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ChenChun\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.
C:\Users\ChenChun\AppData\Local\Temp\ipykernel_26844\137194136.py:102: DtypeWarning: Columns (15,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)


載入資料：409482 筆
正在計算Flesch可讀性指數...
進度: 0/409482 (0.0%)
進度: 1000/409482 (0.2%)
進度: 2000/409482 (0.5%)
進度: 3000/409482 (0.7%)
進度: 4000/409482 (1.0%)
進度: 5000/409482 (1.2%)
進度: 6000/409482 (1.5%)
進度: 7000/409482 (1.7%)
進度: 8000/409482 (2.0%)
進度: 9000/409482 (2.2%)
進度: 10000/409482 (2.4%)
進度: 11000/409482 (2.7%)
進度: 12000/409482 (2.9%)
進度: 13000/409482 (3.2%)
進度: 14000/409482 (3.4%)
進度: 15000/409482 (3.7%)
進度: 16000/409482 (3.9%)
進度: 17000/409482 (4.2%)
進度: 18000/409482 (4.4%)
進度: 19000/409482 (4.6%)
進度: 20000/409482 (4.9%)
進度: 21000/409482 (5.1%)
進度: 22000/409482 (5.4%)
進度: 23000/409482 (5.6%)
進度: 24000/409482 (5.9%)
進度: 25000/409482 (6.1%)
進度: 26000/409482 (6.3%)
進度: 27000/409482 (6.6%)
進度: 28000/409482 (6.8%)
進度: 29000/409482 (7.1%)
進度: 30000/409482 (7.3%)
進度: 31000/409482 (7.6%)
進度: 32000/409482 (7.8%)
進度: 33000/409482 (8.1%)
進度: 34000/409482 (8.3%)
進度: 35000/409482 (8.5%)
進度: 36000/409482 (8.8%)
進度: 37000/409482 (9.0%)
進度: 38000/409482 (9.3%)
進度: 39000/409482 (9.5%)
進度: 40000/409482 (9.

## 類別分類

視覺化

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# 建立資料
data = {
    '類別': ['Books(實體書)', 'Buy a Kindle', 'Toys & Games', 'Amazon Home', 'Office Products',
            'Musical Instruments', 'Arts, Crafts & Sewing', 'Cell Phones & Accessories', 
            'Sports & Outdoors', 'All Electronics', 'AMAZON FASHION', 'Health & Personal Care',
            'Industrial & Scientific', 'Automotive', 'Computers', 'Tools & Home Improvement',
            'Home Audio & Theater', 'Camera & Photo', 'Car Electronics', 'Movies & TV',
            'Portable Audio & Accessories', 'Video Games', 'Amazon Devices', 'All Beauty',
            'GPS & Navigation', 'Fire Phone', 'Baby', 'Pet Supplies', 'Appliances',
            'Apple Products', 'Software', 'Amazon Fire TV', 'Premium Beauty', 'Grocery',
            'Handmade', 'Digital Music', 'Appstore for Android', 'Collectible Coins', 'Sports Collectibles'],
    '資訊透明度': [5,5,4,4,5,3,4,5,4,5,3,3,5,4,5,4,5,5,5,5,5,4,5,3,5,5,3,4,5,5,5,5,3,3,3,5,5,3,3],
    '質量可判斷': [4,5,3,4,5,2,3,5,3,5,2,2,5,3,5,4,5,5,5,4,5,3,5,2,5,5,2,3,5,5,4,5,2,2,2,4,4,2,2],
    '體驗依賴': [2,1,3,2,1,5,3,1,4,1,5,4,1,3,1,2,2,1,1,2,1,4,1,5,1,1,5,4,1,1,2,1,5,5,5,2,2,5,5],
    '評論依賴': [3,2,3,2,2,4,3,2,3,2,4,4,2,3,2,2,3,2,2,3,2,4,2,4,2,2,4,3,2,2,3,2,4,4,4,3,3,4,4],
    '使用後驗證': [2,1,3,2,1,5,3,1,4,1,5,5,1,3,1,2,2,1,1,2,1,4,1,5,1,1,5,4,1,1,2,1,5,5,5,2,2,5,5],
    '最終分類': ['搜尋品','搜尋品','混合型','搜尋品','搜尋品','經驗品','混合型','搜尋品','混合型','搜尋品',
               '經驗品','經驗品','搜尋品','混合型','搜尋品','搜尋品','搜尋品','搜尋品','搜尋品','搜尋品',
               '搜尋品','混合型','搜尋品','經驗品','搜尋品','搜尋品','經驗品','混合型','搜尋品','搜尋品',
               '搜尋品','搜尋品','經驗品','經驗品','經驗品','搜尋品','搜尋品','經驗品','經驗品']
}

df = pd.DataFrame(data)

# 設定字體
plt.rcParams['font.sans-serif'] = ['Arial', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

# 建立視覺化
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 16))

# 1. Radar Chart - Average indicators by classification
categories = ['Search Goods', 'Experience Goods', 'Hybrid']
colors = ['#2E86AB', '#A23B72', '#F18F01']

# Calculate average values for each classification
avg_scores = df.groupby('最終分類')[['資訊透明度', '質量可判斷', '體驗依賴', '評論依賴', '使用後驗證']].mean()

# Map Chinese categories to English
category_mapping = {'搜尋品': 'Search Goods', '經驗品': 'Experience Goods', '混合型': 'Hybrid'}
avg_scores.index = avg_scores.index.map(category_mapping)

# 雷達圖設定
angles = np.linspace(0, 2*np.pi, 5, endpoint=False)
angles = np.concatenate((angles, [angles[0]]))

# 重新建立極座標子圖
ax1 = plt.subplot(221, projection='polar')
ax1.set_theta_offset(np.pi / 2)
ax1.set_theta_direction(-1)

for i, category in enumerate(categories):
    values = avg_scores.loc[category].values
    values = np.concatenate((values, [values[0]]))
    ax1.plot(angles, values, 'o-', linewidth=2, label=category, color=colors[i])
    ax1.fill(angles, values, alpha=0.25, color=colors[i])

ax1.set_xticks(angles[:-1])
ax1.set_xticklabels(['Info Transparency', 'Quality Judgeable', 'Experience Dependent', 'Review Dependent', 'Post-use Verification'])
ax1.set_ylim(0, 5)
ax1.set_title('Average Indicators Radar Chart by Classification', size=14, weight='bold', pad=20)
ax1.legend(loc='upper right', bbox_to_anchor=(1.2, 1.0))
ax1.grid(True)

# 2. Scatter Plot - Info Transparency vs Experience Dependent
category_colors = {'Search Goods': '#2E86AB', 'Experience Goods': '#A23B72', 'Hybrid': '#F18F01'}
# Create English category column for plotting
df['Category_EN'] = df['最終分類'].map(category_mapping)

for category in categories:
    mask = df['Category_EN'] == category
    ax2.scatter(df[mask]['資訊透明度'], df[mask]['體驗依賴'], 
               c=category_colors[category], label=category, alpha=0.7, s=80)

ax2.set_xlabel('Info Transparency', fontsize=12)
ax2.set_ylabel('Experience Dependent', fontsize=12)
ax2.set_title('Info Transparency vs Experience Dependent', fontsize=14, weight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_xlim(0.5, 5.5)
ax2.set_ylim(0.5, 5.5)

# 3. Heatmap - All indicators
metrics = ['資訊透明度', '質量可判斷', '體驗依賴', '評論依賴', '使用後驗證']
metrics_en = ['Info Transparency', 'Quality Judgeable', 'Experience Dependent', 'Review Dependent', 'Post-use Verification']
heatmap_data = df.set_index('類別')[metrics].T

im = ax3.imshow(heatmap_data.values, cmap='RdYlBu_r', aspect='auto', vmin=1, vmax=5)
ax3.set_xticks(range(len(heatmap_data.columns)))
ax3.set_xticklabels(heatmap_data.columns, rotation=45, ha='right', fontsize=8)
ax3.set_yticks(range(len(heatmap_data.index)))
ax3.set_yticklabels(metrics_en, fontsize=10)
ax3.set_title('All Product Categories Rating Heatmap', fontsize=14, weight='bold')

# 添加數值標籤
for i in range(len(heatmap_data.index)):
    for j in range(len(heatmap_data.columns)):
        ax3.text(j, i, int(heatmap_data.iloc[i, j]), ha='center', va='center', 
                color='white' if heatmap_data.iloc[i, j] < 3 else 'black', fontweight='bold')

plt.colorbar(im, ax=ax3, shrink=0.6)

# 4. Bar Chart - Classification count statistics
category_counts = df['Category_EN'].value_counts()
bars = ax4.bar(category_counts.index, category_counts.values, 
               color=[category_colors[cat] for cat in category_counts.index])
ax4.set_title('Product Classification Count Statistics', fontsize=14, weight='bold')
ax4.set_ylabel('Number of Products', fontsize=12)

# 在長條上添加數值
for bar in bars:
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2., height + 0.5,
             f'{int(height)}', ha='center', va='bottom', fontsize=12, fontweight='bold')

plt.tight_layout(pad=3.0)
plt.suptitle('Amazon Product Categories - Five Rating Indicators Visualization', fontsize=16, weight='bold', y=0.98)
plt.show()

# Print classification summary
print("=== Amazon Product Classification Summary ===")
print(f"Search Goods: {len(df[df['最終分類']=='搜尋品'])} categories")
print(f"Experience Goods: {len(df[df['最終分類']=='經驗品'])} categories") 
print(f"Hybrid: {len(df[df['最終分類']=='混合型'])} categories")
print("\nAverage indicators by classification:")
print(avg_scores.round(2))

## 三階段模型
第一階段：資料預處理 - 建立基本變數、對數轉換、虛擬變數等
第二階段：正交化處理 - 對Arousal和Valence進行正交化，避免多重共線性
第三階段：建立交互項 - 創建平方項和交互項
第四階段：準備三個模型 - 參考模型、基礎模型、完整模型
第五階段：執行回歸分析 - 使用statsmodels進行OLS回歸
第六階段：統計檢驗 - 嵌套模型F檢驗
第七階段：結論分析 - 動態判斷最佳模型並提供建議

In [ ]:
import polars as pl
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from scipy import stats

print("="*100)
print("Amazon評論有用性分析 - 完整回歸建模流程")
print("="*100)

# =============================================
# 第一部分：資料預處理和變數建立
# =============================================

print("\n第一階段：資料預處理...")

# 處理評論數據並計算基本屬性
processed_df = df.with_columns([
    # 1. 計算 title_processed 和 text_processed 的總長度
    (pl.col("title_processed").str.len_chars() + pl.col("text_processed").str.len_chars()).alias("total_length"),
    
    # 2. 計算評論發布距今天數（自然對數）
    # unix timestamp 轉換為天數差異再取對數
    ((pl.lit(1750089600000) - pl.col("timestamp")) / 86400000).log().alias("longevity"),
    
    # 3. 將rating轉換為float64
    pl.col("rating").cast(pl.Float64).alias("rating_float"),
    
    # 4. 計算rating_number的自然對數（商品評論數量）
    pl.col("rating_number").log().alias("biz_review_counts_ln")
]).with_columns([
    # 5. 計算每個 user_id 的評論總數
    pl.col("user_id").count().over("user_id").alias("reviewer_posts")
]).with_columns([
    # 6. 計算各種對數變換
    pl.col("total_length").log().alias("length_ln"),
    pl.col("reviewer_posts").log().alias("reviewer_posts_ln"),
    
    # 7. 創建rating的dummy變數（以rating=5為參考組）
    pl.when(pl.col("rating_float") == 1.0).then(1).otherwise(0).alias("rating_1"),
    pl.when(pl.col("rating_float") == 2.0).then(1).otherwise(0).alias("rating_2"),
    pl.when(pl.col("rating_float") == 3.0).then(1).otherwise(0).alias("rating_3"),
    pl.when(pl.col("rating_float") == 4.0).then(1).otherwise(0).alias("rating_4")
    # rating_5 作為參考組，不需要創建
])

# 檢查helpful_vote的分布並決定是否取對數
helpful_vote_stats = processed_df.select([
    pl.col("helpful_vote").mean().alias("mean"),
    pl.col("helpful_vote").median().alias("median"),
    pl.col("helpful_vote").std().alias("std"),
    pl.col("helpful_vote").skew().alias("skewness")
])

print("Helpful_vote 分布統計：")
print(helpful_vote_stats)

# 如果偏度 > 1，表示右偏，需要取對數
skewness_value = helpful_vote_stats.select("skewness").item()
if skewness_value > 1:
    print(f"helpful_vote 右偏 (偏度: {skewness_value:.3f})，將進行對數轉換")
    processed_df = processed_df.with_columns([
        (pl.col("helpful_vote") + 1).log().alias("helpful_vote_ln")  # 加1避免log(0)
    ])
    dependent_var = "helpful_vote_ln"
else:
    print(f"helpful_vote 分布正常 (偏度: {skewness_value:.3f})，不需要對數轉換")
    dependent_var = "helpful_vote"

print(f"依變數確定為: {dependent_var}")

# 正交化處理 Arousal 和 Valence
print("\n執行Arousal和Valence正交化處理...")

# 將 polars DataFrame 轉換為 numpy array 進行正交化
arousal_vals = processed_df.select("Arousal").to_numpy().flatten()
valence_vals = processed_df.select("Valence").to_numpy().flatten()

# 移除缺失值的索引
valid_indices = ~(np.isnan(arousal_vals) | np.isnan(valence_vals))
arousal_clean = arousal_vals[valid_indices]
valence_clean = valence_vals[valid_indices]

# 對 Arousal 進行 Valence 的回歸，取殘差作為正交化的 Arousal
reg_arousal = LinearRegression()
reg_arousal.fit(valence_clean.reshape(-1, 1), arousal_clean)
arousal_residuals = arousal_clean - reg_arousal.predict(valence_clean.reshape(-1, 1))

# 對 Valence 進行 Arousal 的回歸，取殘差作為正交化的 Valence
reg_valence = LinearRegression()
reg_valence.fit(arousal_clean.reshape(-1, 1), valence_clean)
valence_residuals = valence_clean - reg_valence.predict(arousal_clean.reshape(-1, 1))

# 創建正交化結果的完整數組（包含原始的NaN位置）
oArousal = np.full(len(arousal_vals), np.nan)
oValence = np.full(len(valence_vals), np.nan)
oArousal[valid_indices] = arousal_residuals
oValence[valid_indices] = valence_residuals

# 將正交化結果添加到DataFrame
processed_df = processed_df.with_columns([
    pl.Series("oArousal", oArousal),
    pl.Series("oValence", oValence)
])

print("正交化完成！")

# =============================================
# 第二部分：建立交互項和平方項
# =============================================

print("\n第二階段：建立交互項和平方項...")

# 建立交互項和平方項
processed_df = processed_df.with_columns([
    # oArousal的平方項
    (pl.col("oArousal") ** 2).alias("oArousal_squared"),
    
    # oArousal和oValence的交互項
    (pl.col("oArousal") * pl.col("oValence")).alias("oArousal_oValence_interaction")
])

# =============================================
# 第三部分：準備三個階層回歸模型
# =============================================

print("\n第三階段：準備三個階層回歸模型...")

# 準備各模型所需變數
base_controls = [
    "length_ln",           # β1: log(Length)
    "breadth",             # β2: Breadth
    "readability",         # β3: Readability
    "reviewer_posts_ln",   # β4: log(ReviewerPosts)
    "longevity",           # β5: log(Longevity)
    "rating_1",            # β6: ReviewRating dummy
    "rating_2",            # β7: ReviewRating dummy
    "rating_3",            # β8: ReviewRating dummy
    "rating_4",            # β9: ReviewRating dummy
    "biz_review_counts_ln" # β10: log(BizReviewCounts)
]

# Model 1: 參考模型（不包含情感變數）
model1_vars = [dependent_var] + base_controls

# Model 2: 基礎模型（包含線性情感效應）
model2_vars = [dependent_var] + [
    "oValence",            # γ1: oValence
    "oArousal"             # γ2: oArousal
] + base_controls

# Model 3: 完整模型（包含非線性和交互效應）
model3_vars = [dependent_var] + [
    "oValence",                        # γ1: oValence
    "oArousal",                        # γ2: oArousal
    "oArousal_squared",                # γ3: oArousal^2
    "oArousal_oValence_interaction"    # γ4: oArousal*oValence
] + base_controls

# 準備各模型的資料集
df_model1 = processed_df.select(model1_vars).drop_nulls()
df_model2 = processed_df.select(model2_vars).drop_nulls()
df_model3 = processed_df.select(model3_vars).drop_nulls()

print(f"Model 1 觀測數: {df_model1.shape[0]}")
print(f"Model 2 觀測數: {df_model2.shape[0]}")
print(f"Model 3 觀測數: {df_model3.shape[0]}")

# =============================================
# 第四部分：執行回歸分析
# =============================================

# 轉換為pandas進行回歸分析
def run_regression(df, model_name, dependent_var):
    """執行OLS回歸並返回結果"""
    df_pd = df.to_pandas()
    
    # 分離依變數和自變數
    y = df_pd[dependent_var]
    X = df_pd.drop(columns=[dependent_var])
    
    # 加入常數項
    X = sm.add_constant(X)
    
    # 執行回歸
    model = sm.OLS(y, X).fit()
    
    return model, X.columns.tolist()

# 執行三個模型
print("\n" + "="*100)
print("第四階段：執行回歸分析...")
print("="*100)

model1_result, model1_vars_final = run_regression(df_model1, "Model 1", dependent_var)
model2_result, model2_vars_final = run_regression(df_model2, "Model 2", dependent_var)
model3_result, model3_vars_final = run_regression(df_model3, "Model 3", dependent_var)

# =============================================
# 第五部分：結果呈現和分析
# =============================================

# 函數：格式化回歸結果摘要
def print_model_summary(model, model_name, equation):
    """列印模型摘要"""
    print(f"\n{'='*80}")
    print(f"{model_name}")
    print(f"{'='*80}")
    print(f"方程式: {equation}")
    print(f"\n觀測數: {int(model.nobs)}")
    print(f"R-squared: {model.rsquared:.4f}")
    print(f"Adjusted R-squared: {model.rsquared_adj:.4f}")
    print(f"F-statistic: {model.fvalue:.4f} (p-value: {model.f_pvalue:.4e})")
    print(f"AIC: {model.aic:.4f}")
    print(f"BIC: {model.bic:.4f}")
    
    # 顯示關鍵係數
    print(f"\n關鍵係數:")
    print("-" * 80)
    print(f"{'變數':<25} {'係數':<10} {'標準誤':<10} {'t值':<8} {'p值':<10} {'顯著性':<8}")
    print("-" * 80)
    
    for var in model.params.index:
        if var != "const":
            coef = model.params[var]
            se = model.bse[var]
            t_val = model.tvalues[var]
            p_val = model.pvalues[var]
            sig = "***" if p_val < 0.001 else "**" if p_val < 0.01 else "*" if p_val < 0.05 else ""
            print(f"{var:<25} {coef:<10.4f} {se:<10.4f} {t_val:<8.3f} {p_val:<10.4f} {sig:<8}")

# 列印各模型結果
equations = {
    "Model 1": "log(Helpfulness) = α + β1*log(Length) + β2*Breadth + β3*Readability + β4*log(ReviewerPosts) + β5*log(Longevity) + β6~9*ReviewRating + β10*log(BizReviewCounts) + ε",
    "Model 2": "log(Helpfulness) = α + γ1*oValence + γ2*oArousal + β1*log(Length) + β2*Breadth + β3*Readability + β4*log(ReviewerPosts) + β5*log(Longevity) + β6~9*ReviewRating + β10*log(BizReviewCounts) + ε",
    "Model 3": "log(Helpfulness) = α + γ1*oValence + γ2*oArousal + γ3*oArousal² + γ4*oArousal*oValence + β1*log(Length) + β2*Breadth + β3*Readability + β4*log(ReviewerPosts) + β5*log(Longevity) + β6~9*ReviewRating + β10*log(BizReviewCounts) + ε"
}

print_model_summary(model1_result, "Model 1 (參考模型)", equations["Model 1"])
print_model_summary(model2_result, "Model 2 (基礎模型)", equations["Model 2"])
print_model_summary(model3_result, "Model 3 (完整模型)", equations["Model 3"])

# 模型比較表
print(f"\n{'='*100}")
print("模型比較摘要")
print(f"{'='*100}")

comparison_data = {
    "模型": ["Model 1 (參考)", "Model 2 (基礎)", "Model 3 (完整)"],
    "觀測數": [int(model1_result.nobs), int(model2_result.nobs), int(model3_result.nobs)],
    "R²": [f"{model1_result.rsquared:.4f}", f"{model2_result.rsquared:.4f}", f"{model3_result.rsquared:.4f}"],
    "Adj R²": [f"{model1_result.rsquared_adj:.4f}", f"{model2_result.rsquared_adj:.4f}", f"{model3_result.rsquared_adj:.4f}"],
    "AIC": [f"{model1_result.aic:.2f}", f"{model2_result.aic:.2f}", f"{model3_result.aic:.2f}"],
    "BIC": [f"{model1_result.bic:.2f}", f"{model2_result.bic:.2f}", f"{model3_result.bic:.2f}"],
    "F統計量": [f"{model1_result.fvalue:.2f}", f"{model2_result.fvalue:.2f}", f"{model3_result.fvalue:.2f}"]
}

comparison_df = pd.DataFrame(comparison_data)
print(comparison_df.to_string(index=False))

# =============================================
# 第六部分：統計檢驗
# =============================================

# 進行嵌套模型F檢驗
print(f"\n{'='*100}")
print("嵌套模型F檢驗")
print(f"{'='*100}")

def nested_f_test(restricted_model, full_model, model1_name, model2_name):
    """執行嵌套模型F檢驗"""
    rss_restricted = restricted_model.ssr
    rss_full = full_model.ssr
    df_restricted = restricted_model.df_resid
    df_full = full_model.df_resid
    
    # F統計量
    f_stat = ((rss_restricted - rss_full) / (df_restricted - df_full)) / (rss_full / df_full)
    p_value = 1 - stats.f.cdf(f_stat, df_restricted - df_full, df_full)
    
    print(f"\n{model1_name} vs {model2_name}:")
    print(f"F統計量: {f_stat:.4f}")
    print(f"p值: {p_value:.4e}")
    print(f"自由度: ({df_restricted - df_full}, {df_full})")
    significance = "***" if p_value < 0.001 else "**" if p_value < 0.01 else "*" if p_value < 0.05 else "不顯著"
    print(f"結果: {significance}")
    
    return f_stat, p_value

# 執行F檢驗
f1, p1 = nested_f_test(model1_result, model2_result, "Model 1", "Model 2")
f2, p2 = nested_f_test(model2_result, model3_result, "Model 2", "Model 3")
f3, p3 = nested_f_test(model1_result, model3_result, "Model 1", "Model 3")

# =============================================
# 第七部分：研究結論
# =============================================

print(f"\n{'='*100}")
print("研究結論")
print(f"{'='*100}")

# 動態判斷最佳模型
models_results = {
    "Model 1": model1_result,
    "Model 2": model2_result, 
    "Model 3": model3_result
}

# 找出最高Adjusted R²的模型
best_r2_model = max(models_results.keys(), key=lambda x: models_results[x].rsquared_adj)
best_r2_value = models_results[best_r2_model].rsquared_adj

# 找出最低AIC的模型
best_aic_model = min(models_results.keys(), key=lambda x: models_results[x].aic)
best_aic_value = models_results[best_aic_model].aic

print("1. 模型比較結果:")
print(f"   - {best_r2_model}具有最高的解釋力 (Adj R² = {best_r2_value:.4f})")
print(f"   - {best_aic_model}具有最低的AIC值 ({best_aic_value:.2f})，表示最佳的模型適配")

# 檢查AIC和Adj R²是否指向同一個模型
if best_r2_model == best_aic_model:
    print(f"   - 基於Adj R²和AIC準則，{best_aic_model}表現最佳")
else:
    print(f"   - 注意：Adj R²和AIC準則指向不同模型，需進一步考慮研究目的選擇")

print("\n2. 嵌套檢驗結果:")
if p1 < 0.05:
    print("   - 加入線性情感效應顯著改善模型 (Model 2 > Model 1)")
else:
    print("   - 加入線性情感效應未顯著改善模型")

if p2 < 0.05:
    print("   - 加入非線性和交互效應顯著改善模型 (Model 3 > Model 2)")
else:
    print("   - 加入非線性和交互效應未顯著改善模型")

print("\n3. 關鍵發現:")
if 'oArousal_squared' in model3_result.params.index:
    arousal_sq_coef = model3_result.params['oArousal_squared']
    arousal_sq_pval = model3_result.pvalues['oArousal_squared']
    if arousal_sq_pval < 0.05:
        print(f"   - 情感喚醒度存在顯著的非線性效應 (γ3 = {arousal_sq_coef:.4f}, p < 0.05)")
    else:   
        print("   - 情感喚醒度的非線性效應不顯著")

if 'oArousal_oValence_interaction' in model3_result.params.index:
    interaction_coef = model3_result.params['oArousal_oValence_interaction']
    interaction_pval = model3_result.pvalues['oArousal_oValence_interaction']
    if interaction_pval < 0.05:
        print(f"   - 情感效價具有顯著的調節效應 (γ4 = {interaction_coef:.4f}, p < 0.05)")
    else:
        print("   - 情感效價的調節效應不顯著")

print(f"\n4. 最終建議:")
if best_r2_model == best_aic_model:
    print(f"   基於Adj R²和AIC準則的一致性，推薦使用{best_aic_model}作為最終分析模型。")
else:
    print(f"   Adj R²支持{best_r2_model}，AIC支持{best_aic_model}。")
    print(f"   建議根據研究目的選擇：")
    print(f"   - 若重視解釋力最大化：選擇{best_r2_model}")
    print(f"   - 若重視模型簡約性：選擇{best_aic_model}")
    print(f"   - 若理論假設重要：評估F檢驗結果決定是否需要複雜模型")

print(f"\n{'='*100}")
print("分析完成！")
print(f"{'='*100}")

### 倒U型關係驗證

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

print("="*120)
print("倒U型關係驗證 - 遵循 Haans et al. (2016) 方法")
print("="*120)

def validate_inverted_u_relationship(model_result, data_df):
    """
    驗證倒U型關係的完整程序
    參數:
    - model_result: 完整模型(Model 3)的回歸結果
    - data_df: 包含原始數據的DataFrame
    """
    
    # 步驟1: 確認二次項係數顯著且為負
    print("\n" + "="*80)
    print("步驟1: 驗證二次項係數 (γ3) 顯著且為負")
    print("="*80)
    
    # 提取關鍵係數
    gamma2 = model_result.params['oArousal']  # oArousal線性項
    gamma3 = model_result.params['oArousal_squared']  # oArousal平方項
    gamma4 = model_result.params['oArousal_oValence_interaction']  # 交互項
    
    # 提取p值
    p_gamma2 = model_result.pvalues['oArousal']
    p_gamma3 = model_result.pvalues['oArousal_squared']
    p_gamma4 = model_result.pvalues['oArousal_oValence_interaction']
    
    print(f"γ2 (oArousal): {gamma2:.6f} (p = {p_gamma2:.6f})")
    print(f"γ3 (oArousal²): {gamma3:.6f} (p = {p_gamma3:.6f})")
    print(f"γ4 (oArousal×oValence): {gamma4:.6f} (p = {p_gamma4:.6f})")
    
    # 判斷二次項係數
    if gamma3 < 0 and p_gamma3 < 0.05:
        print(f"\n✓ 驗證通過: γ3 = {gamma3:.6f} < 0 且顯著 (p < 0.05)")
        print("  → 支持倒U型關係假設")
        relationship_type = "倒U型"
    elif gamma3 > 0 and p_gamma3 < 0.05:
        print(f"\n✗ γ3 = {gamma3:.6f} > 0 且顯著")
        print("  → 支持U型關係，非倒U型")
        relationship_type = "U型"
    else:
        print(f"\n✗ γ3 = {gamma3:.6f} 不顯著 (p = {p_gamma3:.6f})")
        print("  → 無顯著二次關係")
        relationship_type = "無二次關係"
    
    # 步驟2: 測試斜率在低點和高點的顯著性
    print("\n" + "="*80)
    print("步驟2: 測試斜率在不同oArousal水平的顯著性")
    print("="*80)
    
    # 計算oArousal和oValence的分位數
    arousal_data = data_df['oArousal']
    valence_data = data_df['oValence']
    
    # 定義低、中、高水平
    arousal_low = np.percentile(arousal_data, 10)   # 10th percentile
    arousal_high = np.percentile(arousal_data, 90)  # 90th percentile
    
    valence_low = np.percentile(valence_data, 10)
    valence_med = np.percentile(valence_data, 50)
    valence_high = np.percentile(valence_data, 90)
    
    print(f"oArousal範圍: 低點 = {arousal_low:.3f}, 高點 = {arousal_high:.3f}")
    print(f"oValence水平: 低 = {valence_low:.3f}, 中 = {valence_med:.3f}, 高 = {valence_high:.3f}")
    
    # 斜率計算公式: 斜率 = γ2 + 2γ3*oArousal + γ4*oValence
    print(f"\n斜率公式: 斜率 = {gamma2:.6f} + 2×{gamma3:.6f}×oArousal + {gamma4:.6f}×oValence")
    
    # 計算不同情況下的斜率
    valence_levels = [
        ("低效價", valence_low),
        ("中效價", valence_med), 
        ("高效價", valence_high)
    ]
    
    arousal_levels = [
        ("低喚醒", arousal_low),
        ("高喚醒", arousal_high)
    ]
    
    slope_results = []
    
    print(f"\n{'情感效價':<10} {'喚醒度':<10} {'斜率':<12} {'標準誤':<10} {'t值':<8} {'p值':<10} {'顯著性':<8}")
    print("-" * 80)
    
    for val_name, val_level in valence_levels:
        for aro_name, aro_level in arousal_levels:
            # 計算斜率
            slope = gamma2 + 2 * gamma3 * aro_level + gamma4 * val_level
            
            # 計算斜率的標準誤 (使用Delta方法)
            # Var(slope) = Var(γ2) + 4*aro²*Var(γ3) + val²*Var(γ4) + 
            #              4*aro*Cov(γ2,γ3) + 2*val*Cov(γ2,γ4) + 4*aro*val*Cov(γ3,γ4)
            
            var_gamma2 = model_result.cov_params().loc['oArousal', 'oArousal']
            var_gamma3 = model_result.cov_params().loc['oArousal_squared', 'oArousal_squared']
            var_gamma4 = model_result.cov_params().loc['oArousal_oValence_interaction', 'oArousal_oValence_interaction']
            
            cov_gamma2_gamma3 = model_result.cov_params().loc['oArousal', 'oArousal_squared']
            cov_gamma2_gamma4 = model_result.cov_params().loc['oArousal', 'oArousal_oValence_interaction']
            cov_gamma3_gamma4 = model_result.cov_params().loc['oArousal_squared', 'oArousal_oValence_interaction']
            
            slope_var = (var_gamma2 + 
                        4 * aro_level**2 * var_gamma3 + 
                        val_level**2 * var_gamma4 + 
                        4 * aro_level * cov_gamma2_gamma3 + 
                        2 * val_level * cov_gamma2_gamma4 + 
                        4 * aro_level * val_level * cov_gamma3_gamma4)
            
            slope_se = np.sqrt(slope_var)
            t_stat = slope / slope_se
            p_value = 2 * (1 - stats.t.cdf(abs(t_stat), model_result.df_resid))
            
            significance = "***" if p_value < 0.001 else "**" if p_value < 0.01 else "*" if p_value < 0.05 else ""
            
            print(f"{val_name:<10} {aro_name:<10} {slope:<12.6f} {slope_se:<10.6f} {t_stat:<8.3f} {p_value:<10.6f} {significance:<8}")
            
            slope_results.append({
                'valence_level': val_name,
                'arousal_level': aro_name,
                'slope': slope,
                'se': slope_se,
                't_stat': t_stat,
                'p_value': p_value,
                'significant': p_value < 0.05
            })
    
    # 步驟3: 計算並驗證轉折點
    print("\n" + "="*80)
    print("步驟3: 計算並驗證轉折點位置")
    print("="*80)
    
    print(f"轉折點公式: oArousal* = (-γ2 - γ4×oValence) / (2×γ3)")
    print(f"代入係數: oArousal* = (-{gamma2:.6f} - {gamma4:.6f}×oValence) / (2×{gamma3:.6f})")
    
    turning_points = []
    
    print(f"\n{'情感效價水平':<15} {'轉折點':<12} {'標準誤':<10} {'t值':<8} {'p值':<10} {'顯著性':<8} {'在範圍內':<10}")
    print("-" * 85)
    
    for val_name, val_level in valence_levels:
        # 計算轉折點
        turning_point = (-gamma2 - gamma4 * val_level) / (2 * gamma3)
        
        # 計算轉折點的標準誤 (Delta方法)
        # 對於 f(γ2,γ3,γ4) = (-γ2 - γ4*val) / (2*γ3)
        # 偏導數:
        df_dgamma2 = -1 / (2 * gamma3)
        df_dgamma3 = (gamma2 + gamma4 * val_level) / (2 * gamma3**2)
        df_dgamma4 = -val_level / (2 * gamma3)
        
        # 計算方差
        tp_var = (df_dgamma2**2 * var_gamma2 + 
                 df_dgamma3**2 * var_gamma3 + 
                 df_dgamma4**2 * var_gamma4 + 
                 2 * df_dgamma2 * df_dgamma3 * cov_gamma2_gamma3 + 
                 2 * df_dgamma2 * df_dgamma4 * cov_gamma2_gamma4 + 
                 2 * df_dgamma3 * df_dgamma4 * cov_gamma3_gamma4)
        
        tp_se = np.sqrt(tp_var)
        tp_t_stat = turning_point / tp_se
        tp_p_value = 2 * (1 - stats.t.cdf(abs(tp_t_stat), model_result.df_resid))
        
        # 檢查是否在數據範圍內
        arousal_min = arousal_data.min()
        arousal_max = arousal_data.max()
        in_range = arousal_min <= turning_point <= arousal_max
        
        significance = "***" if tp_p_value < 0.001 else "**" if tp_p_value < 0.01 else "*" if tp_p_value < 0.05 else ""
        range_status = "是" if in_range else "否"
        
        print(f"{val_name:<15} {turning_point:<12.3f} {tp_se:<10.6f} {tp_t_stat:<8.3f} {tp_p_value:<10.6f} {significance:<8} {range_status:<10}")
        
        turning_points.append({
            'valence_level': val_name,
            'turning_point': turning_point,
            'se': tp_se,
            't_stat': tp_t_stat,
            'p_value': tp_p_value,
            'significant': tp_p_value < 0.05,
            'in_range': in_range
        })
    
    print(f"\noArousal數據範圍: [{arousal_min:.3f}, {arousal_max:.3f}]")
    
    # 總結驗證結果
    print("\n" + "="*80)
    print("驗證結果總結")
    print("="*80)
    
    # 檢查斜率模式
    low_slopes_positive = all([r['slope'] > 0 and r['significant'] for r in slope_results if r['arousal_level'] == '低喚醒'])
    high_slopes_negative = all([r['slope'] < 0 and r['significant'] for r in slope_results if r['arousal_level'] == '高喚醒'])
    
    all_turning_points_significant = all([tp['significant'] for tp in turning_points])
    all_turning_points_in_range = all([tp['in_range'] for tp in turning_points])
    
    print("1. 二次項係數檢驗:")
    if gamma3 < 0 and p_gamma3 < 0.05:
        print("   ✓ γ3顯著為負，支持倒U型關係")
    else:
        print("   ✗ γ3不符合倒U型關係要求")
    
    print("\n2. 斜率檢驗:")
    if low_slopes_positive:
        print("   ✓ 低喚醒度時斜率均為正且顯著")
    else:
        print("   ✗ 低喚醒度時斜率不符合預期")
        
    if high_slopes_negative:
        print("   ✓ 高喚醒度時斜率均為負且顯著")
    else:
        print("   ✗ 高喚醒度時斜率不符合預期")
    
    print("\n3. 轉折點檢驗:")
    if all_turning_points_significant:
        print("   ✓ 所有轉折點均顯著")
    else:
        print("   ✗ 部分轉折點不顯著")
        
    if all_turning_points_in_range:
        print("   ✓ 所有轉折點均在數據範圍內")
    else:
        print("   ✗ 部分轉折點超出數據範圍")
    
    # 最終結論
    validation_passed = (gamma3 < 0 and p_gamma3 < 0.05 and 
                        low_slopes_positive and high_slopes_negative and
                        all_turning_points_significant and all_turning_points_in_range)
    
    print(f"\n{'='*80}")
    if validation_passed:
        print("🎉 驗證結論: 倒U型關係得到完全驗證！")
        print("   所有三個驗證步驟均通過，支持以下假設:")
        print("   - 情感喚醒度與評論有用性存在倒U型關係")
        print("   - 情感效價對此倒U型關係具有調節作用")
    else:
        print("⚠️  驗證結論: 倒U型關係未完全得到驗證")
        print("   部分驗證步驟未通過，需要謹慎解釋結果")
    
    return {
        'coefficients': {'gamma2': gamma2, 'gamma3': gamma3, 'gamma4': gamma4},
        'slope_results': slope_results,
        'turning_points': turning_points,
        'validation_passed': validation_passed
    }

# 使用範例 (假設model3_result和final_df已經存在)
validation_results = validate_inverted_u_relationship(model3_result, final_df)

print("\n" + "="*120)
print("使用說明:")
print("請確保已運行前面的回歸分析，然後執行:")
print("validation_results = validate_inverted_u_relationship(model3_result, final_df)")
print("="*120)

第四階層模型(for H2)，因倒U無法驗證因此無用

In [ ]:
print("="*100)
print("Amazon評論有用性分析 - 完整回歸建模流程")
print("="*100)

# =============================================
# 第一部分：資料預處理和變數建立
# =============================================

print("\n第一階段：資料預處理...")

# 處理評論數據並計算基本屬性
processed_df = experience_df.with_columns([
    # 1. 計算 title_processed 和 text_processed 的總長度
    (pl.col("title_processed").str.len_chars() + pl.col("text_processed").str.len_chars()).alias("total_length"),
    
    # 2. 計算評論發布距今天數（自然對數）
    # unix timestamp 轉換為天數差異再取對數
    ((pl.lit(1750089600000) - pl.col("timestamp")) / 86400000).log().alias("longevity"),
    
    # 3. 將rating轉換為float64
    pl.col("rating").cast(pl.Float64).alias("rating_float"),
    
    # 4. 計算rating_number的自然對數（商品評論數量）
    pl.col("rating_number").log().alias("biz_review_counts_ln")
]).with_columns([
    # 5. 計算每個 user_id 的評論總數
    pl.col("user_id").count().over("user_id").alias("reviewer_posts")
]).with_columns([
    # 6. 計算各種對數變換
    pl.col("total_length").log().alias("length_ln"),
    pl.col("reviewer_posts").log().alias("reviewer_posts_ln"),
    
    # 7. 創建rating的dummy變數（以rating=5為參考組）
    pl.when(pl.col("rating_float") == 1.0).then(1).otherwise(0).alias("rating_1"),
    pl.when(pl.col("rating_float") == 2.0).then(1).otherwise(0).alias("rating_2"),
    pl.when(pl.col("rating_float") == 3.0).then(1).otherwise(0).alias("rating_3"),
    pl.when(pl.col("rating_float") == 4.0).then(1).otherwise(0).alias("rating_4")
    # rating_5 作為參考組，不需要創建
])

# 檢查helpful_vote的分布並決定是否取對數
helpful_vote_stats = processed_df.select([
    pl.col("helpful_vote").mean().alias("mean"),
    pl.col("helpful_vote").median().alias("median"),
    pl.col("helpful_vote").std().alias("std"),
    pl.col("helpful_vote").skew().alias("skewness")
])

print("Helpful_vote 分布統計：")
print(helpful_vote_stats)

# 如果偏度 > 1，表示右偏，需要取對數
skewness_value = helpful_vote_stats.select("skewness").item()
if skewness_value > 1:
    print(f"helpful_vote 右偏 (偏度: {skewness_value:.3f})，將進行對數轉換")
    processed_df = processed_df.with_columns([
        (pl.col("helpful_vote") + 1).log().alias("helpful_vote_ln")  # 加1避免log(0)
    ])
    dependent_var = "helpful_vote_ln"
else:
    print(f"helpful_vote 分布正常 (偏度: {skewness_value:.3f})，不需要對數轉換")
    dependent_var = "helpful_vote"

print(f"依變數確定為: {dependent_var}")

# 正交化處理 Arousal 和 Valence
print("\n執行Arousal和Valence正交化處理...")

# 將 polars DataFrame 轉換為 numpy array 進行正交化
arousal_vals = processed_df.select("Arousal").to_numpy().flatten()
valence_vals = processed_df.select("Valence").to_numpy().flatten()

# 移除缺失值的索引
valid_indices = ~(np.isnan(arousal_vals) | np.isnan(valence_vals))
arousal_clean = arousal_vals[valid_indices]
valence_clean = valence_vals[valid_indices]

# 對 Arousal 進行 Valence 的回歸，取殘差作為正交化的 Arousal
reg_arousal = LinearRegression()
reg_arousal.fit(valence_clean.reshape(-1, 1), arousal_clean)
arousal_residuals = arousal_clean - reg_arousal.predict(valence_clean.reshape(-1, 1))

# 對 Valence 進行 Arousal 的回歸，取殘差作為正交化的 Valence
reg_valence = LinearRegression()
reg_valence.fit(arousal_clean.reshape(-1, 1), valence_clean)
valence_residuals = valence_clean - reg_valence.predict(arousal_clean.reshape(-1, 1))

# 創建正交化結果的完整數組（包含原始的NaN位置）
oArousal = np.full(len(arousal_vals), np.nan)
oValence = np.full(len(valence_vals), np.nan)
oArousal[valid_indices] = arousal_residuals
oValence[valid_indices] = valence_residuals

# 將正交化結果添加到DataFrame
processed_df = processed_df.with_columns([
    pl.Series("oArousal", oArousal),
    pl.Series("oValence", oValence)
])

print("正交化完成！")

# =============================================
# 第二部分：建立交互項和平方項
# =============================================

print("\n第二階段：建立交互項和平方項...")

# 建立交互項和平方項
processed_df = processed_df.with_columns([
    # oArousal的平方項
    (pl.col("oArousal") ** 2).alias("oArousal_squared"),
    
    # oArousal和oValence的交互項
    (pl.col("oArousal") * pl.col("oValence")).alias("oArousal_oValence_interaction"),
    
    # 新增：oValence和oArousal²的交互項 (γ5項)
    (pl.col("oValence") * (pl.col("oArousal") ** 2)).alias("oValence_oArousal_squared_interaction")
])

# =============================================
# 第三部分：準備四個階層回歸模型
# =============================================

print("\n第三階段：準備四個階層回歸模型...")

# 準備各模型所需變數
base_controls = [
    "length_ln",           # β1: log(Length)
    "breadth",             # β2: Breadth
    "readability",         # β3: Readability
    "reviewer_posts_ln",   # β4: log(ReviewerPosts)
    "longevity",           # β5: log(Longevity)
    "rating_1",            # β6: ReviewRating dummy
    "rating_2",            # β7: ReviewRating dummy
    "rating_3",            # β8: ReviewRating dummy
    "rating_4",            # β9: ReviewRating dummy
    "biz_review_counts_ln" # β10: log(BizReviewCounts)
]

# Model 1: 參考模型（不包含情感變數）
model1_vars = [dependent_var] + base_controls

# Model 2: 基礎模型（包含線性情感效應）
model2_vars = [dependent_var] + [
    "oValence",            # γ1: oValence
    "oArousal"             # γ2: oArousal
] + base_controls

# Model 3: 包含非線性效應的模型
model3_vars = [dependent_var] + [
    "oValence",                        # γ1: oValence
    "oArousal",                        # γ2: oArousal
    "oArousal_squared",                # γ3: oArousal^2
    "oArousal_oValence_interaction"    # γ4: oArousal*oValence
] + base_controls

# Model 4: 完整模型（包含所有交互項）- 方程式(4)
model4_vars = [dependent_var] + [
    "oValence",                                # γ1: oValence
    "oArousal",                                # γ2: oArousal
    "oArousal_squared",                        # γ3: oArousal^2
    "oArousal_oValence_interaction",           # γ4: oArousal*oValence
    "oValence_oArousal_squared_interaction"    # γ5: oValence*oArousal^2
] + base_controls

# 準備各模型的資料集
df_model1 = processed_df.select(model1_vars).drop_nulls()
df_model2 = processed_df.select(model2_vars).drop_nulls()
df_model3 = processed_df.select(model3_vars).drop_nulls()
df_model4 = processed_df.select(model4_vars).drop_nulls()

print(f"Model 1 觀測數: {df_model1.shape[0]}")
print(f"Model 2 觀測數: {df_model2.shape[0]}")
print(f"Model 3 觀測數: {df_model3.shape[0]}")
print(f"Model 4 觀測數: {df_model4.shape[0]}")

# =============================================
# 第四部分：執行回歸分析
# =============================================

# 轉換為pandas進行回歸分析
def run_regression(df, model_name, dependent_var):
    """執行OLS回歸並返回結果"""
    df_pd = df.to_pandas()
    
    # 分離依變數和自變數
    y = df_pd[dependent_var]
    X = df_pd.drop(columns=[dependent_var])
    
    # 加入常數項
    X = sm.add_constant(X)
    
    # 執行回歸
    model = sm.OLS(y, X).fit()
    
    return model, X.columns.tolist()

# 執行四個模型
print("\n" + "="*100)
print("第四階段：執行回歸分析...")
print("="*100)

model1_result, model1_vars_final = run_regression(df_model1, "Model 1", dependent_var)
model2_result, model2_vars_final = run_regression(df_model2, "Model 2", dependent_var)
model3_result, model3_vars_final = run_regression(df_model3, "Model 3", dependent_var)
model4_result, model4_vars_final = run_regression(df_model4, "Model 4", dependent_var)

# =============================================
# 第五部分：結果呈現和分析
# =============================================

# 函數：格式化回歸結果摘要
def print_model_summary(model, model_name, equation):
    """列印模型摘要"""
    print(f"\n{'='*80}")
    print(f"{model_name}")
    print(f"{'='*80}")
    print(f"方程式: {equation}")
    print(f"\n觀測數: {int(model.nobs)}")
    print(f"R-squared: {model.rsquared:.4f}")
    print(f"Adjusted R-squared: {model.rsquared_adj:.4f}")
    print(f"F-statistic: {model.fvalue:.4f} (p-value: {model.f_pvalue:.4e})")
    print(f"AIC: {model.aic:.4f}")
    print(f"BIC: {model.bic:.4f}")
    
    # 顯示關鍵係數
    print(f"\n關鍵係數:")
    print("-" * 80)
    print(f"{'變數':<35} {'係數':<10} {'標準誤':<10} {'t值':<8} {'p值':<10} {'顯著性':<8}")
    print("-" * 80)
    
    for var in model.params.index:
        if var != "const":
            coef = model.params[var]
            se = model.bse[var]
            t_val = model.tvalues[var]
            p_val = model.pvalues[var]
            sig = "***" if p_val < 0.001 else "**" if p_val < 0.01 else "*" if p_val < 0.05 else ""
            print(f"{var:<35} {coef:<10.4f} {se:<10.4f} {t_val:<8.3f} {p_val:<10.4f} {sig:<8}")

# 列印各模型結果
equations = {
    "Model 1": "log(Helpfulness) = α + β1*log(Length) + β2*Breadth + β3*Readability + β4*log(ReviewerPosts) + β5*log(Longevity) + β6~9*ReviewRating + β10*log(BizReviewCounts) + ε",
    "Model 2": "log(Helpfulness) = α + γ1*oValence + γ2*oArousal + β1*log(Length) + β2*Breadth + β3*Readability + β4*log(ReviewerPosts) + β5*log(Longevity) + β6~9*ReviewRating + β10*log(BizReviewCounts) + ε",
    "Model 3": "log(Helpfulness) = α + γ1*oValence + γ2*oArousal + γ3*oArousal² + γ4*oArousal*oValence + β1*log(Length) + β2*Breadth + β3*Readability + β4*log(ReviewerPosts) + β5*log(Longevity) + β6~9*ReviewRating + β10*log(BizReviewCounts) + ε",
    "Model 4": "log(Helpfulness) = α + γ1*oValence + γ2*oArousal + γ3*oArousal² + γ4*oArousal*oValence + γ5*oValence*oArousal² + β1*log(Length) + β2*Breadth + β3*Readability + β4*log(ReviewerPosts) + β5*log(Longevity) + β6~9*ReviewRating + β10*log(BizReviewCounts) + ε"
}

print_model_summary(model1_result, "Model 1 (參考模型)", equations["Model 1"])
print_model_summary(model2_result, "Model 2 (基礎模型)", equations["Model 2"])
print_model_summary(model3_result, "Model 3 (非線性模型)", equations["Model 3"])
print_model_summary(model4_result, "Model 4 (完整模型 - 方程式4)", equations["Model 4"])

# 模型比較表
print(f"\n{'='*100}")
print("模型比較摘要")
print(f"{'='*100}")

comparison_data = {
    "模型": ["Model 1 (參考)", "Model 2 (基礎)", "Model 3 (非線性)", "Model 4 (完整)"],
    "觀測數": [int(model1_result.nobs), int(model2_result.nobs), int(model3_result.nobs), int(model4_result.nobs)],
    "R²": [f"{model1_result.rsquared:.4f}", f"{model2_result.rsquared:.4f}", f"{model3_result.rsquared:.4f}", f"{model4_result.rsquared:.4f}"],
    "Adj R²": [f"{model1_result.rsquared_adj:.4f}", f"{model2_result.rsquared_adj:.4f}", f"{model3_result.rsquared_adj:.4f}", f"{model4_result.rsquared_adj:.4f}"],
    "AIC": [f"{model1_result.aic:.2f}", f"{model2_result.aic:.2f}", f"{model3_result.aic:.2f}", f"{model4_result.aic:.2f}"],
    "BIC": [f"{model1_result.bic:.2f}", f"{model2_result.bic:.2f}", f"{model3_result.bic:.2f}", f"{model4_result.bic:.2f}"],
    "F統計量": [f"{model1_result.fvalue:.2f}", f"{model2_result.fvalue:.2f}", f"{model3_result.fvalue:.2f}", f"{model4_result.fvalue:.2f}"]
}

comparison_df = pd.DataFrame(comparison_data)
print(comparison_df.to_string(index=False))

# =============================================
# 第六部分：統計檢驗
# =============================================

# 進行嵌套模型F檢驗
print(f"\n{'='*100}")
print("嵌套模型F檢驗")
print(f"{'='*100}")

def nested_f_test(restricted_model, full_model, model1_name, model2_name):
    """執行嵌套模型F檢驗"""
    rss_restricted = restricted_model.ssr
    rss_full = full_model.ssr
    df_restricted = restricted_model.df_resid
    df_full = full_model.df_resid
    
    # F統計量
    f_stat = ((rss_restricted - rss_full) / (df_restricted - df_full)) / (rss_full / df_full)
    p_value = 1 - stats.f.cdf(f_stat, df_restricted - df_full, df_full)
    
    print(f"\n{model1_name} vs {model2_name}:")
    print(f"F統計量: {f_stat:.4f}")
    print(f"p值: {p_value:.4e}")
    print(f"自由度: ({df_restricted - df_full}, {df_full})")
    significance = "***" if p_value < 0.001 else "**" if p_value < 0.01 else "*" if p_value < 0.05 else "不顯著"
    print(f"結果: {significance}")
    
    return f_stat, p_value

# 執行F檢驗
f1, p1 = nested_f_test(model1_result, model2_result, "Model 1", "Model 2")
f2, p2 = nested_f_test(model2_result, model3_result, "Model 2", "Model 3")
f3, p3 = nested_f_test(model3_result, model4_result, "Model 3", "Model 4")
f4, p4 = nested_f_test(model1_result, model4_result, "Model 1", "Model 4")

# =============================================
# 第七部分：研究結論
# =============================================

print(f"\n{'='*100}")
print("研究結論")
print(f"{'='*100}")

# 動態判斷最佳模型
models_results = {
    "Model 1": model1_result,
    "Model 2": model2_result, 
    "Model 3": model3_result,
    "Model 4": model4_result
}

# 找出最高Adjusted R²的模型
best_r2_model = max(models_results.keys(), key=lambda x: models_results[x].rsquared_adj)
best_r2_value = models_results[best_r2_model].rsquared_adj

# 找出最低AIC的模型
best_aic_model = min(models_results.keys(), key=lambda x: models_results[x].aic)
best_aic_value = models_results[best_aic_model].aic

print("1. 模型比較結果:")
print(f"   - {best_r2_model}具有最高的解釋力 (Adj R² = {best_r2_value:.4f})")
print(f"   - {best_aic_model}具有最低的AIC值 ({best_aic_value:.2f})，表示最佳的模型適配")

# 檢查AIC和Adj R²是否指向同一個模型
if best_r2_model == best_aic_model:
    print(f"   - 基於Adj R²和AIC準則，{best_aic_model}表現最佳")
else:
    print(f"   - 注意：Adj R²和AIC準則指向不同模型，需進一步考慮研究目的選擇")

print("\n2. 嵌套檢驗結果:")
if p1 < 0.05:
    print("   - 加入線性情感效應顯著改善模型 (Model 2 > Model 1)")
else:
    print("   - 加入線性情感效應未顯著改善模型")

if p2 < 0.05:
    print("   - 加入非線性和交互效應顯著改善模型 (Model 3 > Model 2)")
else:
    print("   - 加入非線性和交互效應未顯著改善模型")

if p3 < 0.05:
    print("   - 加入oValence*oArousal²交互項顯著改善模型 (Model 4 > Model 3)")
else:
    print("   - 加入oValence*oArousal²交互項未顯著改善模型")

print("\n3. 關鍵發現:")
if 'oArousal_squared' in model4_result.params.index:
    arousal_sq_coef = model4_result.params['oArousal_squared']
    arousal_sq_pval = model4_result.pvalues['oArousal_squared']
    if arousal_sq_pval < 0.05:
        print(f"   - 情感喚醒度存在顯著的非線性效應 (γ3 = {arousal_sq_coef:.4f}, p < 0.05)")
    else:   
        print("   - 情感喚醒度的非線性效應不顯著")

if 'oArousal_oValence_interaction' in model4_result.params.index:
    interaction_coef = model4_result.params['oArousal_oValence_interaction']
    interaction_pval = model4_result.pvalues['oArousal_oValence_interaction']
    if interaction_pval < 0.05:
        print(f"   - oArousal*oValence交互效應顯著 (γ4 = {interaction_coef:.4f}, p < 0.05)")
    else:
        print("   - oArousal*oValence交互效應不顯著")

if 'oValence_oArousal_squared_interaction' in model4_result.params.index:
    valence_arousal_sq_coef = model4_result.params['oValence_oArousal_squared_interaction']
    valence_arousal_sq_pval = model4_result.pvalues['oValence_oArousal_squared_interaction']
    if valence_arousal_sq_pval < 0.05:
        print(f"   - oValence*oArousal²交互效應顯著 (γ5 = {valence_arousal_sq_coef:.4f}, p < 0.05)")
        print("   - 這表示情感效價會調節情緒喚醒度的倒U型關係")
    else:
        print("   - oValence*oArousal²交互效應不顯著")

print(f"\n4. 方程式(4)的理論意義:")
print(f"   - γ5項 (oValence*oArousal²) 代表情感效價對情緒喚醒度倒U型關係的調節作用")
print(f"   - 正的γ5係數表示：正面情感會加強高喚醒度對有用性的負面影響")
print(f"   - 負的γ5係數表示：正面情感會緩解高喚醒度對有用性的負面影響")

print(f"\n5. 最終建議:")
if best_r2_model == best_aic_model:
    print(f"   基於Adj R²和AIC準則的一致性，推薦使用{best_aic_model}作為最終分析模型。")
else:
    print(f"   Adj R²支持{best_r2_model}，AIC支持{best_aic_model}。")
    print(f"   建議根據研究目的選擇：")
    print(f"   - 若重視解釋力最大化：選擇{best_r2_model}")
    print(f"   - 若重視模型簡約性：選擇{best_aic_model}")
    print(f"   - 若理論假設重要：評估F檢驗結果決定是否需要複雜模型")

print(f"\n{'='*100}")
print("分析完成！包含方程式(4)的完整交互項分析")
print(f"{'='*100}")

轉折點移動分析(for H2)

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats

def analyze_turning_point_sensitivity(model_result, data_df):
    """
    分析轉折點隨情感效價變化的移動程度
    
    轉折點公式: oArousal* = (-γ2 - γ4×oValence) / (2×γ3)
    對oValence的一階導數: d(oArousal*)/d(oValence) = -γ4 / (2×γ3)
    
    參數:
    - model_result: 完整模型(Model 3)的回歸結果
    - data_df: 包含原始數據的DataFrame
    """
    
    print("="*100)
    print("轉折點移動程度分析 - 情感效價敏感性")
    print("="*100)
    
    # 提取回歸係數
    gamma2 = model_result.params['oArousal']  # oArousal線性項
    gamma3 = model_result.params['oArousal_squared']  # oArousal平方項  
    gamma4 = model_result.params['oArousal_oValence_interaction']  # 交互項
    
    # 提取p值
    p_gamma2 = model_result.pvalues['oArousal']
    p_gamma3 = model_result.pvalues['oArousal_squared']
    p_gamma4 = model_result.pvalues['oArousal_oValence_interaction']
    
    print(f"模型係數:")
    print(f"γ2 (oArousal): {gamma2:.6f} (p = {p_gamma2:.6f})")
    print(f"γ3 (oArousal²): {gamma3:.6f} (p = {p_gamma3:.6f})")
    print(f"γ4 (oArousal×oValence): {gamma4:.6f} (p = {p_gamma4:.6f})")
    
    # 計算轉折點對效價的敏感性
    print(f"\n轉折點公式: oArousal* = (-γ2 - γ4×oValence) / (2×γ3)")
    print(f"轉折點公式: oArousal* = (-{gamma2:.6f} - {gamma4:.6f}×oValence) / (2×{gamma3:.6f})")
    
    # 一階導數計算
    sensitivity = -gamma4 / (2 * gamma3)
    
    print(f"\n" + "="*80)
    print("轉折點敏感性分析")
    print("="*80)
    
    print(f"一階導數公式: d(oArousal*)/d(oValence) = -γ4 / (2×γ3)")
    print(f"一階導數計算: d(oArousal*)/d(oValence) = -{gamma4:.6f} / (2×{gamma3:.6f})")
    print(f"敏感性係數: {sensitivity:.6f}")
    
    # 敏感性的標準誤計算 (Delta方法)
    # 對於 f(γ3,γ4) = -γ4 / (2*γ3)
    # 偏導數:
    df_dgamma3 = gamma4 / (2 * gamma3**2)
    df_dgamma4 = -1 / (2 * gamma3)
    
    # 取得方差和協方差
    var_gamma3 = model_result.cov_params().loc['oArousal_squared', 'oArousal_squared']
    var_gamma4 = model_result.cov_params().loc['oArousal_oValence_interaction', 'oArousal_oValence_interaction']
    cov_gamma3_gamma4 = model_result.cov_params().loc['oArousal_squared', 'oArousal_oValence_interaction']
    
    # 計算敏感性的方差
    sensitivity_var = (df_dgamma3**2 * var_gamma3 + 
                      df_dgamma4**2 * var_gamma4 + 
                      2 * df_dgamma3 * df_dgamma4 * cov_gamma3_gamma4)
    
    sensitivity_se = np.sqrt(sensitivity_var)
    sensitivity_t_stat = sensitivity / sensitivity_se
    sensitivity_p_value = 2 * (1 - stats.t.cdf(abs(sensitivity_t_stat), model_result.df_resid))
    
    # 敏感性顯著性
    if sensitivity_p_value < 0.001:
        significance = "***"
    elif sensitivity_p_value < 0.01:
        significance = "**"
    elif sensitivity_p_value < 0.05:
        significance = "*"
    else:
        significance = ""
    
    print(f"\n敏感性統計檢驗:")
    print(f"標準誤: {sensitivity_se:.6f}")
    print(f"t統計量: {sensitivity_t_stat:.3f}")
    print(f"p值: {sensitivity_p_value:.6f} {significance}")
    
    # 敏感性解釋
    print(f"\n" + "="*80)
    print("敏感性解釋")
    print("="*80)
    
    if sensitivity_p_value < 0.05:
        print(f"✓ 轉折點敏感性顯著 (p < 0.05)")
        
        if sensitivity > 0:
            direction = "正向移動"
            explanation = "情感效價增加時，轉折點向更高的喚醒度移動"
        else:
            direction = "負向移動"
            explanation = "情感效價增加時，轉折點向更低的喚醒度移動"
            
        print(f"移動方向: {direction}")
        print(f"解釋: {explanation}")
        print(f"移動幅度: 情感效價每增加1個單位，轉折點移動 {abs(sensitivity):.6f} 個喚醒度單位")
    else:
        print(f"✗ 轉折點敏感性不顯著 (p = {sensitivity_p_value:.6f})")
        print("轉折點不隨情感效價顯著變化")
    
    # 數據範圍分析
    arousal_data = data_df['oArousal']
    valence_data = data_df['oValence']
    
    arousal_range = arousal_data.max() - arousal_data.min()
    valence_range = valence_data.max() - valence_data.min()
    
    print(f"\n數據範圍:")
    print(f"oArousal範圍: [{arousal_data.min():.4f}, {arousal_data.max():.4f}] (寬度: {arousal_range:.4f})")
    print(f"oValence範圍: [{valence_data.min():.4f}, {valence_data.max():.4f}] (寬度: {valence_range:.4f})")
    
    # 實際移動幅度計算
    if sensitivity_p_value < 0.05:
        max_movement = abs(sensitivity) * valence_range
        relative_movement = max_movement / arousal_range * 100
        
        print(f"\n實際移動分析:")
        print(f"在整個效價範圍內，轉折點最大移動距離: {max_movement:.4f} 個喚醒度單位")
        print(f"相對於喚醒度總範圍的移動比例: {relative_movement:.2f}%")
        
        if relative_movement > 50:
            movement_level = "大幅移動"
        elif relative_movement > 20:
            movement_level = "中等移動"  
        elif relative_movement > 10:
            movement_level = "小幅移動"
        else:
            movement_level = "微小移動"
            
        print(f"移動程度評估: {movement_level}")
    
    # 不同效價水平下的具體轉折點計算
    print(f"\n" + "="*80)
    print("不同效價水平下的轉折點位置")
    print("="*80)
    
    # 定義效價水平
    valence_levels = [
        ("低效價 (10%)", np.percentile(valence_data, 10)),
        ("中低效價 (25%)", np.percentile(valence_data, 25)),
        ("中效價 (50%)", np.percentile(valence_data, 50)),
        ("中高效價 (75%)", np.percentile(valence_data, 75)),
        ("高效價 (90%)", np.percentile(valence_data, 90))
    ]
    
    print(f"{'效價水平':<15} {'效價值':<10} {'轉折點':<12} {'相對變化':<12}")
    print("-" * 55)
    
    baseline_turning_point = None
    
    for i, (val_name, val_level) in enumerate(valence_levels):
        # 計算轉折點
        turning_point = (-gamma2 - gamma4 * val_level) / (2 * gamma3)
        
        if i == 0:  # 設定基準點
            baseline_turning_point = turning_point
            relative_change = 0.0
        else:
            relative_change = turning_point - baseline_turning_point
            
        print(f"{val_name:<15} {val_level:<10.4f} {turning_point:<12.4f} {relative_change:<+12.4f}")
    
    # 總結
    print(f"\n" + "="*80)
    print("總結")
    print("="*80)
    
    if sensitivity_p_value < 0.05:
        print(f"✓ 轉折點顯著隨情感效價變化")
        print(f"  - 敏感性係數: {sensitivity:.6f} {significance}")
        print(f"  - 移動方向: {'正向' if sensitivity > 0 else '負向'}")
        if sensitivity_p_value < 0.05:
            print(f"  - 在效價範圍內最大移動: {max_movement:.4f} 個單位 ({relative_movement:.2f}%)")
            print(f"  - 移動程度: {movement_level}")
    else:
        print(f"✗ 轉折點不隨情感效價顯著變化")
        print(f"  - 敏感性係數不顯著 (p = {sensitivity_p_value:.6f})")
    
    return {
        'sensitivity': sensitivity,
        'sensitivity_se': sensitivity_se,
        'sensitivity_t_stat': sensitivity_t_stat,
        'sensitivity_p_value': sensitivity_p_value,
        'significant': sensitivity_p_value < 0.05,
        'max_movement': max_movement if sensitivity_p_value < 0.05 else 0,
        'relative_movement_percent': relative_movement if sensitivity_p_value < 0.05 else 0,
        'turning_points_by_valence': valence_levels
    }

# 使用範例
# sensitivity_results = analyze_turning_point_sensitivity(model3_result, df_model3)

In [ ]:

import pandas as pd
# 讀取原始 CSV 檔案
df = pd.read_csv('amazon_2023_with_breadth_readability.csv')

In [ ]:

import pandas as pd
# 讀取原始 CSV 檔案
df = pd.read_csv('amazon_2023_with_breadth_readability.csv')